In [1]:
import tensorflow as tf
import numpy as np
import gym
from collections import deque
import random
import warnings
warnings.filterwarnings('ignore')

# Test the game

In [5]:
env = gym.make('SpaceInvaders-v0')
env.reset()
for _ in range(3000):
    env.step(env.action_space.sample())
    env.render('human')
env.close()
